In [36]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error

In [2]:
#import modelos
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

# Procesamiento de los datos

In [3]:
#filtrando los datos
df = pd.read_csv('train.csv', dtype={
    'tipodepropiedad':'category', 'ciudad':'category',\
    'provincia':'category'}, parse_dates=[16])

df_test = pd.read_csv('test.csv', dtype={
    'tipodepropiedad':'category', 'ciudad':'category',\
    'provincia':'category'}, parse_dates=[16])

### Manejo de valores nulos

In [4]:
df.isnull().sum()

id                                 0
titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

A los datos del dataframe que tienen una de las columnas (de 'metrostotales' y 'metroscubiertos') pero no la otra, copiamos el dato de una a la otra según sea necesario.

In [5]:
## funcion para setear los superficies
def set_metros(row):
    total = row.metrostotales
    covered = row.metroscubiertos
    if np.isnan(total):
        row.metrostotales = covered
        return row
    if np.isnan(covered):
        row.metroscubiertos = total
        return row
    return row

In [6]:
df[['metrostotales', 'metroscubiertos']] = df[['metrostotales', 'metroscubiertos']].apply(set_metros, axis = 1)
df_test[['metrostotales', 'metroscubiertos']] = df_test[['metrostotales', 'metroscubiertos']].apply(set_metros, axis = 1)

In [47]:
df = df.loc[~df['tipodepropiedad'].isnull()] #Elimino los datos que no tengan tipo de propiedad (46)
df['garages'] = df['garages'].fillna(value = 0)


KeyError: 'tipodepropiedad'

In [48]:
df_test['garages'] = df_test['garages'].fillna(value = 0)

In [8]:
num_cols_con_nulls = ['antiguedad', 'habitaciones', 'banos', 'idzona']
cat_cols_con_nulls = ['ciudad', 'provincia']

In [9]:
# define numerical imputer
num_imputer = SimpleImputer(strategy='mean')

# imputing on numerical data
df[num_cols_con_nulls] = num_imputer.fit_transform(df[num_cols_con_nulls])
df_test[num_cols_con_nulls] = num_imputer.fit_transform(df_test[num_cols_con_nulls])

In [10]:
from sklearn.base import TransformerMixin

class SeriesImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        If the Series is of dtype Object, then impute with the most frequent object.
        If the Series is not of dtype Object, then impute with the mean.  

        """
    def fit(self, X, y=None):
        self.fill = X.value_counts().index[0]
        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [11]:
# define categorical imputer
cat_imputer = SeriesImputer()

# imputing on categorical data
df['ciudad'] = cat_imputer.fit_transform(df['ciudad'])
df['provincia'] = cat_imputer.fit_transform(df['provincia'])

df_test['ciudad'] = cat_imputer.fit_transform(df_test['ciudad'])
df_test['provincia'] = cat_imputer.fit_transform(df_test['provincia'])

In [12]:
df.isnull().sum()

id                                 0
titulo                          5387
descripcion                     1615
tipodepropiedad                    0
direccion                      53028
ciudad                             0
provincia                          0
antiguedad                         0
habitaciones                       0
garages                            0
banos                              0
metroscubiertos                    0
metrostotales                      0
idzona                             0
lat                           123457
lng                           123457
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

## Encoding variables categóricas

### ID_Zona

Efectuare un ordinal encoding, teniendo en cuenta los precios promedio por id_zona obtenidos a partir del TP1

In [13]:
df.groupby('idzona').agg({'tipodepropiedad' : 'count'}).mean()

tipodepropiedad    12.168053
dtype: float64

In [14]:
#Agrego precio por metro cuadrado
df['precioporm2'] = df['precio'] / df['metrostotales']

In [15]:
precio_m2_por_idzona = df.groupby('idzona').agg({'precioporm2' : 'mean'})
precio_m2_por_idzona.reset_index(inplace = True)
precio_m2_por_idzona = precio_m2_por_idzona.sort_values(by = 'precioporm2')
precio_m2_por_idzona

,idzona,precioporm2
18431,129693.0,720.000000
17020,114109.0,781.093596
5348,41576.0,785.714286
13186,80278.0,800.000000
6789,50076.0,800.000000
...,...,...
15642,100074.0,330496.250000
2113,11340.0,396641.700000
9822,65194.0,400000.000000
2474,15954.0,455555.555556


In [16]:
def cambiar_valores(x):
    i = 0
    if x.name == 'precioporm2':
        i+=1
        return i
    return x

precio_m2_por_idzona = precio_m2_por_idzona.apply(cambiar_valores)
precio_m2_por_idzona['precioporm2'] = precio_m2_por_idzona['precioporm2'].cumsum()
precio_m2_por_idzona

,idzona,precioporm2
18431,129693.0,1
17020,114109.0,2
5348,41576.0,3
13186,80278.0,4
6789,50076.0,5
...,...,...
15642,100074.0,19716
2113,11340.0,19717
9822,65194.0,19718
2474,15954.0,19719


In [17]:
orden_idzona = precio_m2_por_idzona.set_index('idzona').to_dict()
orden_idzona = orden_idzona['precioporm2']

In [18]:
df['idzona_ordinal'] = df.idzona.map(orden_idzona)
df_test['idzona_ordinal'] = df_test.idzona.map(orden_idzona)
df.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,precioporm2,idzona_ordinal
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8.116273,2.0,1.0,...,NaN,2015-08-23,0.0,0.0,0.0,0.0,0.0,2273000.0,28412.500000,19199
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.000000,3.0,2.0,...,-99.227655,2013-06-28,0.0,0.0,0.0,1.0,1.0,3600000.0,20000.000000,18523
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.000000,3.0,2.0,...,NaN,2015-10-17,0.0,0.0,0.0,0.0,0.0,1200000.0,7228.915663,10390
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.000000,2.0,1.0,...,-99.688015,2012-03-09,0.0,0.0,0.0,1.0,1.0,650000.0,9701.492537,12561
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.000000,2.0,1.0,...,NaN,2016-06-07,0.0,0.0,0.0,0.0,0.0,1150000.0,12105.263158,16128


### Tipo de propiedad

In [19]:
df['tipodepropiedad'].unique()

[Apartamento, Casa en condominio, Casa, Terreno, Terreno comercial, ..., Terreno industrial, Huerta, Lote, Hospedaje, Garage]
Length: 24
Categories (24, object): [Apartamento, Casa en condominio, Casa, Terreno, ..., Huerta, Lote, Hospedaje, Garage]

In [20]:
df.shape

(239954, 25)

In [21]:
#One Hot Encoding
df = pd.get_dummies(df, prefix = ['Tipo'], columns = ['tipodepropiedad'])
df_test = pd.get_dummies(df_test, prefix = ['Tipo'], columns = ['tipodepropiedad'])

In [22]:
df.shape

(239954, 48)

In [23]:
df.head()

,id,titulo,descripcion,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,...,Tipo_Oficina comercial,Tipo_Otros,Tipo_Quinta Vacacional,Tipo_Rancho,Tipo_Terreno,Tipo_Terreno comercial,Tipo_Terreno industrial,Tipo_Villa,Tipo_Hospedaje,Tipo_Garage
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8.116273,2.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.000000,3.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Urbi Tonala,Tonalá,Jalisco,5.000000,3.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.000000,2.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,PASEOS DEL SOL,Zapopan,Jalisco,10.000000,2.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0


### Provincia

Efectuare un ordinal encoding, teniendo en cuenta los precios promedio por provincia obtenidos a partir del TP1

In [24]:
precio_promedio_por_provincia = pd.read_csv('provincias_por_precio')
precio_promedio_por_provincia = precio_promedio_por_provincia.sort_values(by = 'precio_por_m2_mean')



def cambiar_valores(x):
    i = 0
    if x.name == 'precio_por_m2_mean':
        i+=1
        return i
    return x

precio_promedio_por_provincia = precio_promedio_por_provincia.apply(cambiar_valores)
precio_promedio_por_provincia['precio_por_m2_mean'] = precio_promedio_por_provincia['precio_por_m2_mean'].cumsum()
precio_promedio_por_provincia

,provincia,precio_por_m2_mean
31,Baja California,1
30,Durango,2
29,Sonora,3
28,Baja California Sur,4
27,Coahuila,5
26,Tlaxcala,6
25,Chihuahua,7
24,Aguascalientes,8
23,Colima,9
22,Hidalgo,10


In [25]:
orden_provincias = precio_promedio_por_provincia.set_index('provincia').to_dict()
orden_provincias = orden_provincias['precio_por_m2_mean']
orden_provincias

{'Baja California': 1,
 'Durango': 2,
 'Sonora': 3,
 'Baja California Sur': 4,
 'Coahuila': 5,
 'Tlaxcala': 6,
 'Chihuahua': 7,
 'Aguascalientes': 8,
 'Colima': 9,
 'Hidalgo': 10,
 'Tamaulipas': 11,
 'Campeche': 12,
 'Yucatan': 13,
 'Sinaloa': 14,
 'Michoacan': 15,
 'Zacatecas': 16,
 'Nayarit': 17,
 'Oaxaca': 18,
 'San Luis Potosi': 19,
 'Guanajuato': 20,
 'Queretaro': 21,
 'Veracruz': 22,
 'Morelos': 23,
 'Chiapas': 24,
 'Puebla': 25,
 'Quintana Roo': 26,
 'Tabasco': 27,
 'Jalisco': 28,
 'Nuevo Leon': 29,
 'Mexico': 30,
 'Guerrero': 31,
 'Distrito Federal': 32}

In [26]:
df['provincia'] = df['provincia'].replace({'Edo. de México' : 'Mexico',
                                               'Querétaro' : 'Queretaro',
                                               'Nuevo León' : 'Nuevo Leon',
                                               'San luis Potosí' : 'San Luis Potosi',
                                               'Yucatán' : 'Yucatan',
                                               'Baja California Norte' : 'Baja California',
                                               'Michoacán' : 'Michoacan'})
df_test['provincia'] = df_test['provincia'].replace({'Edo. de México' : 'Mexico',
                                               'Querétaro' : 'Queretaro',
                                               'Nuevo León' : 'Nuevo Leon',
                                               'San luis Potosí' : 'San Luis Potosi',
                                               'Yucatán' : 'Yucatan',
                                               'Baja California Norte' : 'Baja California',
                                               'Michoacán' : 'Michoacan'})

In [27]:
df['provincia_ordinal'] = df.provincia.map(orden_provincias)
df_test['provincia_ordinal'] = df.provincia.map(orden_provincias)

df.head()

,id,titulo,descripcion,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,...,Tipo_Otros,Tipo_Quinta Vacacional,Tipo_Rancho,Tipo_Terreno,Tipo_Terreno comercial,Tipo_Terreno industrial,Tipo_Villa,Tipo_Hospedaje,Tipo_Garage,provincia_ordinal
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8.116273,2.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,32
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.000000,3.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,32
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Urbi Tonala,Tonalá,Jalisco,5.000000,3.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,28
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Mexico,1.000000,2.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,30
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,PASEOS DEL SOL,Zapopan,Jalisco,10.000000,2.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,28


In [28]:
#df['ciudad'] = df['ciudad'].cat.codes
#df['provincia'] = df['provincia'].cat.codes
df = df.drop(columns = ['provincia', 'ciudad'])
df_test = df_test.drop(columns = ['provincia', 'ciudad'])

In [29]:
df.head()

,id,titulo,descripcion,direccion,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,...,Tipo_Otros,Tipo_Quinta Vacacional,Tipo_Rancho,Tipo_Terreno,Tipo_Terreno comercial,Tipo_Terreno industrial,Tipo_Villa,Tipo_Hospedaje,Tipo_Garage,provincia_ordinal
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Avenida Division del Norte 2005,8.116273,2.0,1.0,2.0,80.0,80.0,...,0,0,0,0,0,0,0,0,0,32
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",AV. MEXICO,10.000000,3.0,2.0,2.0,268.0,180.0,...,0,0,0,0,0,0,0,0,0,32
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Urbi Tonala,5.000000,3.0,2.0,2.0,144.0,166.0,...,0,0,0,0,0,0,0,0,0,28
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,IGNACIO MANUEL ALTAMIRANO 128,1.000000,2.0,1.0,1.0,63.0,67.0,...,0,0,0,0,0,0,0,0,0,30
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,PASEOS DEL SOL,10.000000,2.0,1.0,1.0,95.0,95.0,...,0,0,0,0,0,0,0,0,0,28


In [30]:
df['fecha'] = pd.to_datetime(df['fecha'])

y = df['precio'].values

df['dia'] = df['fecha'].dt.day
df['mes'] = df['fecha'].dt.month
df['anio'] = df['fecha'].dt.year

df = df.drop(columns=['fecha', 'id', 'idzona', 'titulo', 'descripcion', 'direccion', 'lat', 'lng', 'precio']).copy()

df['cant_extras'] = sum([df['gimnasio'],df['usosmultiples'],df['piscina'],
                        df['escuelascercanas'],df['centroscomercialescercanos']])

df.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,gimnasio,usosmultiples,piscina,escuelascercanas,...,Tipo_Terreno comercial,Tipo_Terreno industrial,Tipo_Villa,Tipo_Hospedaje,Tipo_Garage,provincia_ordinal,dia,mes,anio,cant_extras
0,8.116273,2.0,1.0,2.0,80.0,80.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,32,23,8,2015,0.0
1,10.000000,3.0,2.0,2.0,268.0,180.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,32,28,6,2013,2.0
2,5.000000,3.0,2.0,2.0,144.0,166.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,28,17,10,2015,0.0
3,1.000000,2.0,1.0,1.0,63.0,67.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,30,9,3,2012,2.0
4,10.000000,2.0,1.0,1.0,95.0,95.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,28,7,6,2016,0.0


In [32]:
df_test['fecha'] = pd.to_datetime(df_test['fecha'])

df_test['dia'] = df_test['fecha'].dt.day
df_test['mes'] = df_test['fecha'].dt.month
df_test['anio'] = df_test['fecha'].dt.year

df_test = df_test.drop(columns=['fecha', 'id', 'idzona', 'titulo', 'descripcion', 'direccion', 'lat', 'lng']).copy()


df_test['cant_extras'] = sum([df_test['gimnasio'],df_test['usosmultiples'],df_test['piscina'],
                        df_test['escuelascercanas'],df_test['centroscomercialescercanos']])

df_test.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,gimnasio,usosmultiples,piscina,escuelascercanas,...,Tipo_Terreno comercial,Tipo_Terreno industrial,Tipo_Villa,Tipo_Huerta,Tipo_Lote,provincia_ordinal,dia,mes,anio,cant_extras
0,29.000000,3.0,NaN,4.0,300.0,300.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,32.0,20,7,2013,0.0
1,8.152599,1.0,1.0,1.0,67.0,67.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,32.0,24,10,2015,0.0
2,0.000000,2.0,1.0,2.0,87.0,100.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,28.0,30,5,2015,1.0
3,2.000000,2.0,2.0,2.0,86.0,86.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,30.0,2,4,2015,0.0
4,10.000000,2.0,1.0,1.0,80.0,76.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,28.0,15,8,2013,2.0


## Ejecucion de modelos

In [49]:
X = df.copy().values

#splitear train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

test_entrega = df_test.copy()
test_entrega.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,gimnasio,usosmultiples,piscina,escuelascercanas,...,Tipo_Terreno comercial,Tipo_Terreno industrial,Tipo_Villa,Tipo_Huerta,Tipo_Lote,provincia_ordinal,dia,mes,anio,cant_extras
0,29.000000,3.0,0.0,4.0,300.0,300.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,32.0,20,7,2013,0.0
1,8.152599,1.0,1.0,1.0,67.0,67.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,32.0,24,10,2015,0.0
2,0.000000,2.0,1.0,2.0,87.0,100.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,28.0,30,5,2015,1.0
3,2.000000,2.0,2.0,2.0,86.0,86.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,30.0,2,4,2015,0.0
4,10.000000,2.0,1.0,1.0,80.0,76.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,28.0,15,8,2013,2.0


In [39]:
#randomforest
random_forest = RandomForestRegressor(n_estimators=200, max_depth=1000, n_jobs=-1)
random_forest.fit(X_train, y_train)
precio_pred=random_forest.predict(X_test)
mean_absolute_error(precio_pred, y_test)

6534.813457310784

In [50]:
precio_pred=random_forest.predict(test_entrega)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

# LO QUE SIGUE ES PARA PROBAR TODOS LOS MODELOS!

In [ ]:
def ejecutar_modelos(X_train, y_train, X_test, y_test, modelos, nombres):
    resultado = []
    for nombre,modelo in zip(modelos_nombre, modelos):
        print('------------------------------------------------')
        print('Comienza a entrenar: '+ nombre)
        print()
        modelo = modelo   
        modelo.fit(X_train,y_train)
        score = modelo.score(X_test,y_test)
        resultado.append((str(nombre), str(score)))
        print('**FINALIZADO** ' + nombre + ' Score: ' + str(score))
        print()
    return resultado

In [ ]:
#splitear train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
modelos=[LinearRegression(),KNeighborsRegressor(),Ridge(),Lasso(),MLPRegressor(alpha=20),DecisionTreeRegressor(),ExtraTreeRegressor(),XGBRegressor(),RandomForestRegressor(),AdaBoostRegressor(),GradientBoostingRegressor(),BaggingRegressor()]
modelos_nombre=['LinearRegression','KNNRegressor','Ridge','Lasso','MLPRegressor','DecisionTree','ExtraTree','XGBoost','RandomForest','AdaBoost','GradientBoost','Bagging']
score_=[]

In [ ]:
resultado_normal = ejecutar_modelos(X_train, y_train, X_test, y_test, modelos, modelos_nombre)

In [ ]:
#aplico logaritmo
y_log = np.log(y)
X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.25, random_state=42)

In [131]:
resultado_log = ejecutar_modelos(X_train, y_train_log, X_test, y_test_log, modelos, modelos_nombre)

------------------------------------------------
Comienza a entrenar: LinearRegression

**FINALIZADO** LinearRegression Score: 0.8694456738348924

------------------------------------------------
Comienza a entrenar: KNNRegressor

**FINALIZADO** KNNRegressor Score: 0.9482154731694089

------------------------------------------------
Comienza a entrenar: Ridge

**FINALIZADO** Ridge Score: 0.8694428224027243

------------------------------------------------
Comienza a entrenar: Lasso

**FINALIZADO** Lasso Score: 0.8291433244618149

------------------------------------------------
Comienza a entrenar: MLPRegressor

**FINALIZADO** MLPRegressor Score: -3423.666033807385

------------------------------------------------
Comienza a entrenar: DecisionTree

**FINALIZADO** DecisionTree Score: 0.9997621697246022

------------------------------------------------
Comienza a entrenar: ExtraTree

**FINALIZADO** ExtraTree Score: 0.9864591997239792

------------------------------------------------
Comi

In [ ]:
#estanderizar datos
X_estandar=StandardScaler().fit_transform(X)
y=np.array(y).reshape(-1,1)
y_estandar=StandardScaler().fit_transform(y)
y_estandar=y_estandar.ravel()
X_train_estandar, X_test_estandar, y_train_estandar, y_test_estandar = train_test_split(X_estandar, y_estandar, test_size=0.25, random_state=42)


In [ ]:
resultado_estandar = ejecutar_modelos(X_train_estandar, y_train_estandar, X_test_estandar, y_test_estandar, modelos, modelos_nombre)

In [133]:
#Crea un archivo csv de la forma nombre_del_algoritmo, score

#Columnas: string con nombres de columnas separados por comas
#Resultados: lista de resultados, siendo cada resultado una lista conformada por tuplas (Nombre_algoritmo, Score)
def crear_csv_resultados(columnas, resultados, nombre_archivo):
    with open(nombre_archivo, 'w') as f:
        f.write(columnas + '\n')
        lineas_a_escribir = []
        for i in range(len(resultados[0])):
            linea = resultados[0][i][0] + ','
            for resultado in resultados:
                linea = linea + resultado[i][1] + ','
            linea = linea[:len(linea)-1]
            lineas_a_escribir.append(linea)
        for linea in lineas_a_escribir:
            f.write(linea + '\n')

In [134]:
NOMBRE_CSV = 'one_hot+ordinal_provincias+ordinal_idzona' # CAMBIARLO ASI NO SE PISAN!!!!
crear_csv_resultados('nombre,normal,log,estandar',[resultado_normal, resultado_log, resultado_estandar],NOMBRE_CSV)

In [135]:
pd.read_csv('one_hot_ciudades')

FileNotFoundError: [Errno 2] File b'one_hot_ciudades' does not exist: b'one_hot_ciudades'

In [136]:
pd.read_csv('num_imputter_mean')

FileNotFoundError: [Errno 2] File b'num_imputter_mean' does not exist: b'num_imputter_mean'

In [137]:
pd.read_csv(NOMBRE_CSV)

,nombre,normal,log,estandar
0,LinearRegression,0.784459,0.869446,NaN
1,KNNRegressor,0.930493,0.948215,NaN
2,Ridge,0.784466,0.869443,NaN
3,Lasso,0.784461,0.829143,NaN
4,MLPRegressor,0.995827,-3423.666034,NaN
5,DecisionTree,0.999190,0.999762,NaN
6,ExtraTree,0.982291,0.986459,NaN
7,XGBoost,0.995595,0.996998,NaN
8,RandomForest,0.999771,0.999910,NaN
9,AdaBoost,0.796962,0.940996,NaN
